In [3]:
import re
from groq import Groq
import os
from dotenv import load_dotenv

_ = load_dotenv()

In [29]:
client = Groq(api_key = os.getenv('GROQ_API_KEY'))
# MODEL = 'llama3-70b-8192'
MODEL= 'llama-3.3-70b-versatile'

In [25]:
chat_completion = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": "Hello world"}]
)

In [26]:
chat_completion.choices[0].message.content

'<think>\n\n</think>\n\nHello! How can I assist you today? 😊'

In [12]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model=MODEL, 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [13]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [14]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [15]:
abot = Agent(prompt)

In [16]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the dog's weight using average_dog_weight
Action: average_dog_weight: Toy Poodle
PAUSE


In [17]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [18]:
next_prompt = "Observation: {}".format(result)
abot(next_prompt)

'Answer: A toy poodle weighs 7 lbs.'

In [19]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

## Compiling all of this in unified function

In [30]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [31]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [32]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: To find the combined weight of the two dogs, I need to find the average weight of a Border Collie and a Scottish Terrier. I can use the average_dog_weight action to get the weights of each breed and then add them together.

Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now that I have the average weight of a Border Collie, I need to find the average weight of a Scottish Terrier. I can use the average_dog_weight action again to get the weight of a Scottish Terrier.

Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: Now that I have the average weights of both breeds, I can add them together to find the combined weight of the two dogs.

Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Thought: The calculation is complete, and I have the co

### Creating a Langgraph agent

In [34]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults

In [36]:
tool = TavilySearchResults(max_results=2) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [37]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [38]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [40]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model=MODEL,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=1
)

In [41]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

# model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(llm, [tool], system=prompt)

In [43]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather today'}, 'id': 'call_ffy0', 'type': 'tool_call'}
Back to the model!


In [44]:
result['messages'][-1].content

'The current weather in San Francisco is partly cloudy with a temperature of 53.1°F (11.7°C) and a feels-like temperature of 49.5°F (9.7°C). The wind is blowing at 11.0 mph (17.6 kph) from the northwest, and the humidity is 86%.'

In [55]:
# From the results, it looks like all the tool calls were done parallely for year 2024 as the model in the 2nd tool call does not have any input about Super Bowl 2024 winner

query = "Who won the super bowl in 2021? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

abot = Agent(llm, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2021 winner'}, 'id': 'call_0s2t', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Tampa Bay Buccaneers headquarters location'}, 'id': 'call_kqzp', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Florida'}, 'id': 'call_avhe', 'type': 'tool_call'}
Back to the model!


In [57]:
query = "Who won the super bowl in 2021? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

In [56]:
print(result['messages'][-1].content)

The winner of the Super Bowl in 2021 was the Tampa Bay Buccaneers. The Tampa Bay Buccaneers' headquarters is located in the state of Florida. The GDP of Florida is approximately $1.28 trillion.


In [67]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
question_prompt_template=ChatPromptTemplate.from_template("Who won the super bowl in {year}? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question.")

In [ ]:
chain = question_prompt_template | RunnableLambda(lambda x: {"messages": x.messages})

In [69]:
question_prompt_template.invoke({"year":2022}) | 

ChatPromptValue(messages=[HumanMessage(content='Who won the super bowl in 2022? In what state is the winning team headquarters located? What is the GDP of that state? Answer each question.', additional_kwargs={}, response_metadata={})])